<a href="https://colab.research.google.com/github/choarauc/form/blob/main/BoundingBox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('face_images.csv')
df.insert(1, 'p', 1)
df.insert(2, 'xmoy', (df.xmax + df.xmin)/2)
df.insert(3, 'ymoy', (df.ymax + df.ymin)/2)
df.insert(4, 'w', (df.xmax - df.xmin))
df.insert(5, 'h', (df.ymax - df.ymin))

df.head()

 fonctions load_image et show_bounding_box.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
folder_images = 'images/'

@tf.function
def load_image(filepath, resize=None):
    im = tf.io.read_file(folder_images + filepath)
    im = tf.image.decode_png(im, channels=3)
    if resize:
        return tf.image.resize(im, resize)
    else :
        return im


def show_bounding_box(im, bbox, normalised=True, color='r'):
    # Signification de bbox
    x, y, w, h = bbox
    # Convertir les cordonées (x,y,w,h) en (x1,x2,y1,y2)
    x1=x-w/2
    x2=x+w/2
    y1=y-h/2
    y2=y+h/2
    
    # redimensionner en cas de normalisation
    if normalised:
        x1=x1*im.shape[1]
        x2=x2*im.shape[1]
        y1=y1*im.shape[0]
        y2=y2*im.shape[0]
        
    # Afficher l'image
    plt.imshow(im)
    
    # Afficher la bounding box
    plt.plot([x1,x2,x2,x1,x1],[y1,y1,y2,y2,y1],"r")
        
im = load_image(df.filename[0])

bbox = df[['xmoy', 'ymoy', 'w', 'h']].values[0]
# Afficher l'image ainsi que la bounding box
show_bounding_box(im, bbox)

In [ ]:
idx=1

im = load_image(df.filename[idx])
plt.imshow(im)
liste_colonns = ['x'+str(i) for i in range(68)] + ['y'+str(i) for i in range(68)]
landmarks = df[liste_colonns].values[idx]
plt.plot(landmarks[:68]*im.shape[1],landmarks[68:]*im.shape[0], '.',color='red')

Distribution des valeurs


---



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.displot(df.x0)
plt.show()
sns.displot(df.y0)
plt.show()
sns.displot(df.x33)
plt.show()
sns.displot(df.y33)
plt.show()

generer des images sans visage

In [ ]:
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm

# Return all path in folder iccv09Data with .jpg extention
backgrounds_dir = './iccv09Data/'
background_image_files = glob(backgrounds_dir+'**.jpg')

background_image = []
for p in tqdm(background_image_files):
    img = plt.imread(p)
    background_image.append(img)

len(background_image)


def random_crop(img, WINDOW_SIZE):
    max_allowed_size = np.min(img.shape[:2])
    if WINDOW_SIZE>max_allowed_size:
        return random_crop(img, int(WINDOW_SIZE/1.5))
    else :
        size = np.random.randint(WINDOW_SIZE, max_allowed_size)
    max_width = img.shape[0] - size - 1
    max_height = img.shape[1] - size - 1
    left = 0 if (max_width <= 1)  else np.random.randint(0, max_width)
    top  = 0 if (max_height <= 1) else np.random.randint(0, max_height)
    return img[left:left+size,top:top+size]

def open_background(img: str, resize=(256,256), WINDOW_SIZE: int=150):
    img = random_crop(img, WINDOW_SIZE)
    if resize:
        img = cv2.resize(img, resize)
    return img

import random
def sample_data(n: int):
    xs = [open_background(f, WINDOW_SIZE=70) for f in np.random.choice(background_image, n, replace=True)]
    ys = np.zeros([n, y_train.shape[1]])
    return np.array(xs), ys

Séparer le jeu de données df.filepath et les variables cibles en un ensemble d'entraînement X_train_path, y_train, et en un ensemble de test X_test_path, y_test.

In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

X_train_path, X_test_path, y_train, y_test = train_test_split(df.filename, df.drop(['filename', 'xmin', 'xmax', 'ymin', 'ymax'], axis=1), train_size=0.95, random_state=1234)

# X_test = []
# for p in tqdm(X_test_path):
#     im = plt.imread(p)
#     if (im.shape==2):
#         im = np.expand_dims(im, axis=-1)
#         im = np.concatenate([im, im, im], axis=-1)
#     im = im*255
#     im = im[...,[0,1,2]].astype(np.uint8)
#     X_test.append(cv2.resize(im, (256,256)))
    
# X_test = np.array(X_test)

dataset tensorflow permettant de charger les images d'entraînement en mémoires.

In [ ]:
batch_size = 64
ratio = 1/2

def load_image(filepath, y, resize=(256,256)):
    im = tf.io.read_file(folder_images + filepath)
    im = tf.image.decode_png(im, channels=3)
#     im_shape = tf.shape(im)
    im = tf.image.resize(im, resize)
    tx_max = 256*tf.nn.relu(y[1]-y[3]/2)
    tx_min = -256*tf.nn.relu(1-y[1]-y[3]/2)
    ty_max = 256*tf.nn.relu(y[2]-y[4]/2)
    ty_min = -256*tf.nn.relu(1-y[2]-y[4]/2)
    tx = np.random.uniform(tx_min, tx_max)
    ty = np.random.uniform(ty_min, ty_max)
    im = tf.keras.preprocessing.image.apply_affine_transform(
    im.numpy(), theta=0, tx=ty, ty=tx, shear=0, zx=1, zy=1, row_axis=0, col_axis=1,
    channel_axis=2, fill_mode='nearest', cval=0.0, order=1
)
    y_new = y.numpy().copy()
    y_new[1] += -tx/256
    y_new[2] += -ty/256
    y_new[5:5+68] += -tx/256
    y_new[5+68:] += -ty/256
    
    return im, y_new


dataset_train = tf.data.Dataset.from_tensor_slices((X_train_path, y_train))

dataset_train = dataset_train.shuffle(100000).map(
    lambda x, y : tf.py_function(
        load_image,
        [x, y],
        [tf.float32, tf.float32]
    ),
    num_parallel_calls=-1).
batch(int(batch_size * ratio))

In [ ]:
#afficher les premieres images 

In [ ]:
filepath = X_train_path.values[1]
bbox = y_train.values[1]
img, y = tf.py_function(load_image, [filepath, bbox], [tf.float32, tf.float32])
plt.figure(figsize=(12,5))
plt.subplot(121)
show_bounding_box(img.numpy().astype(int), y[1:5])
landmarks = y[5:]
plt.plot(landmarks[:68]*img.shape[1],landmarks[68:]*img.shape[0], '.',color='red')

generateur de batchs avec ou sans objet

In [ ]:
def generator(X, y, batch_size, ratio=0.5):
    batch_face = int(batch_size*ratio)
    while(True):
        iterator_img = iter(dataset_train)
        for i in range(0, len(X)-batch_face, batch_face):
            data_X, data_Y = next(iterator_img)
            noobject, y_nooboject = sample_data(batch_size-batch_face)
            data_X = np.concatenate([data_X, noobject], axis=0)
            data_Y = np.concatenate([data_Y, y_nooboject], axis=0)
            yield data_X, data_Y
            
gen = generator(X_train_path, y_train, batch_size, ratio=ratio)

Définition du modèle de détection d'objet.

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU, Flatten, Input
from tensorflow.keras.models import Model, Sequential, load_model

# Load the model efficientNet
efficientNet = EfficientNetB0(include_top=False, input_shape=(256,256,3))

# Freeze the blackbone
for layer in efficientNet.layers:
    layer.trainable = False

input_model = Input(shape=[256,256,3])
feature = efficientNet(input_model)
x = GlobalAveragePooling2D()(feature)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
regression = Dense(4, activation='linear')(x)
object_prob = Dense(1, activation='sigmoid')(x)
key_points = Dense(68*2, activation='linear')(x)
output_model = tf.concat([object_prob, regression, key_points], axis=-1)

model = Model(inputs=input_model, outputs=output_model)
model.summary()

Fonction de perte

In [ ]:
lambda_landmarks = 10
lambda_regression = 100
lambda_classification = 1

def loss_landmarks(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred), axis=-1)

def loss_regression(y_true, y_pred):
    return (tf.square(y_true[:,0]-y_pred[:,0]) + tf.square(y_true[:,1]-y_pred[:,1]) + tf.square(tf.sqrt(tf.abs(y_true[:,2])) - tf.sqrt(tf.abs(y_pred[:,2]))) + tf.square(tf.sqrt(tf.abs(y_true[:,3])) - tf.sqrt(tf.abs(y_pred[:,3]))))


def loss_function(y_true, y_pred):
    return lambda_regression*y_true[:,0]*loss_regression(y_true[:,1:5], y_pred[:,1:5]) + lambda_landmarks*y_true[:,0]*loss_landmarks(y_true[:,5:], y_pred[:,5:]) + lambda_classification*tf.keras.losses.BinaryCrossentropy()(y_true[:,0], y_pred[:,0])

model.compile(loss= loss_function, optimizer='adam')

In [ ]:
model.fit_generator(gen, steps_per_epoch=int(len(X_train_path)/(batch_size*ratio)), epochs=10)# , validation_data=(X_test, y_test)

In [ ]:
import time
import cv2, urllib, json

def show_img(img, model):
    plt.imshow(img/255)
    t0=time.time()
    prediction = model.predict(np.array([img], dtype=np.float32))[0]
    p, x0, y0, w0, h0 = prediction[:5]
    landmarks = prediction[5:]
    plt.plot(landmarks[:68]*img.shape[1],landmarks[68:]*img.shape[0], '.',color='red')
    print(x0, y0, w0, h0)
    print("Execution time :",time.time()-t0,"secondes")
    print('Probability :', p)
    x1= (x0-w0/2)*img.shape[1]
    x2= (x0+w0/2)*img.shape[1]
    y1= (y0-h0/2)*img.shape[0]
    y2= (y0+h0/2)*img.shape[0]
    plt.plot([x1,x2,x2,x1,x1], [y1,y1,y2,y2,y1], "r")
    plt.show()
    
def url_to_image(url):
    resp = urllib.request.urlopen(url) 
    img = np.asarray(bytearray(resp.read()), dtype="uint8")
    img = cv2.imdecode(img, -1)
    img = cv2.resize(img, (256,256))
    img = img[..., [2,1,0]]
    return tf.keras.applications.efficientnet.preprocess_input(img)

## Example :
img = url_to_image("https://images.sudouest.fr/2020/10/02/5f7729c066a4bd5078d0c6dd/golden/1200x750/donald-trump-a-annonce.jpg")
show_img(img , model)
plt.show()


test de webcam

In [ ]:
%matplotlib inline
from IPython.display import HTML
from PIL import Image
import base64, io
import numpy as np

main_text = """
<style type="text/css">
    canvas {
        display: none;
    }
    </style>

<video id="video" width="320" height="240" autoplay></video>
<button id="snap">Snap Photo</button>
<canvas id="canvas" width="320" height="240"></canvas>

<script>
// Grab elements, create settings, etc.
var video = document.getElementById('video');

// Get access to the camera!
if(navigator.mediaDevices && navigator.mediaDevices.getUserMedia) {
    // Not adding `{ audio: true }` since we only want video now
    navigator.mediaDevices.getUserMedia({ video: true }).then(function(stream) {
        //video.src = window.URL.createObjectURL(stream);
        //video.play();
        video.srcObject=stream;
        video.play();
    });
}

// Elements for taking the snapshot
var canvas = document.getElementById('canvas');
var context = canvas.getContext('2d');
var video = document.getElementById('video');
var image = canvas.toDataURL("image/png");
IPython.notebook.kernel.execute("image = '" + image + "'")

// Trigger photo take
document.getElementById("snap").addEventListener("click", function() {
	context.drawImage(video, 0, 0, 320, 240);
    var myCanvas = document.getElementById('canvas');
    var image = myCanvas.toDataURL("image/png");
    IPython.notebook.kernel.execute("print('testing')")
    IPython.notebook.kernel.execute("imageWebCam = '" + image + "'")
});
</script>

"""

def show_bboxes_webcam(model):
    img = np.array(Image.open(io.BytesIO(base64.b64decode(imageWebCam.split(',')[1]))))[:,:,0:3]
    img = tf.image.resize(img, (256,256))
    show_img(img, model)
    
HTML(main_text)

Une fois le boutton "Snap Photo" pressé, exécuter la cellule suivante.


In [ ]:
show_bboxes_webcam(model)

test sur camera

In [ ]:
import cv2
import numpy as np

# To capture video from webcam. 
cap = cv2.VideoCapture(0)


font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale              = 1
fontColor              = (0,255,0)
lineType               = 2


while True:
    
    try :
        # Read the frame
        _, img = cap.read()
        img = img

        img_resized = cv2.resize(img, (256, 256))

        prediction = model(np.array([img_resized[:,:,[2,1,0]]], dtype=np.float32))[0]
        obj, x_m, y_m, w, h = prediction[:5]
        landmarks = prediction[5:]
        point_x = landmarks[:68]*img.shape[1]
        point_y = landmarks[68:]*img.shape[0]
        for i in range(68):
            cv2.circle(img, (point_x[i],point_y[i]), radius=0, color=(0, 255, 0), thickness=3)
        x_m = (x_m * img.shape[1]).numpy()
        y_m = (y_m * img.shape[0]).numpy()
        w = (tf.abs(w) * img.shape[1]).numpy()
        h = (h * img.shape[0]).numpy()
        # Draw the rectangle around each face
        cv2.rectangle(img, (int(x_m-w/2), int(y_m-h/2)), (int(x_m+w/2), int(y_m+h/2)), (0, 255, 0), 2)
        cv2.putText(img, str(obj.numpy()), 
                        (int(x_m-w/2), int(y_m-h/2)+30), 
                        font, 
                        fontScale,
                        fontColor,
                        lineType)

        # Display
        cv2.imshow('img', img)
        # Stop if escape key is pressed
        k = cv2.waitKey(30) & 0xff
        if k==27:
            break
    except :
        break
# Release the VideoCapture object
cap.release()